In [287]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm

In [149]:
def calc_miss_chance(df_path: Path) -> float:
    df_orig = pd.read_csv(df_path, index_col=None)
    df_orig = df_orig.drop(['1','2','3','5','6','7'], axis=1)
    df_orig['4'] = df_orig['4'].str.replace('S0010', 'S010')
    df_orig['8'] = df_orig['8'].str.replace('S0010', 'S010')

    df = pd.DataFrame()
#     df['score'] = df_orig['0'].astype(np.float)
    df['phone_actual'] = df_orig['4'].str[6:7].astype(np.int)
    df['id_actual'] = df_orig['4'].str[9:22]
#     df['class_actual'] = df_orig['4'].str[9:13].astype(np.int)
#     df['distinct_actual'] = df_orig['4'].str[15:17].astype(np.int)
#     df['sample_actual'] = df_orig['4'].str[19:22].astype(np.int)
    df['angle_actual'] = df_orig['4'].str[28:31].astype(np.int)
    df['index_actual'] = df_orig['4'].str[38:].astype(np.int)

#     df['phone_predicted'] = df_orig['8'].str[6:7].astype(np.int)
    df['id_predicted'] = df_orig['8'].str[9:22]
#     df['class_predicted'] = df_orig['8'].str[9:13].astype(np.int)
#     df['distinct_predicted'] = df_orig['8'].str[15:17].astype(np.int)
#     df['sample_predicted'] = df_orig['8'].str[19:22].astype(np.int)
    
    df['id_match'] = df['id_actual'] == df['id_predicted']
    df_acc = df[['id_actual', 'phone_actual', 'angle_actual', 'index_actual', 'id_predicted', 'id_match']]
    df_vc = df_acc.groupby(['id_actual', 'phone_actual', 'angle_actual', 'id_predicted'])['id_match'].value_counts()
    df_vc = df_vc.rename_axis(['id_actual', 'phone_actual', 'angle_actual', 'id_predicted', 'id_match']).to_frame('counts').reset_index()
    total_samples = df_vc.set_index(['id_actual', 'phone_actual', 'angle_actual']).index.unique().__len__()
    nomiss_samples = df_vc[df_vc.id_match == True].index.__len__()
    miss_chance = 100 - nomiss_samples / (total_samples) * 100
    return miss_chance



In [157]:
for df_path in Path('../../pipeline_results/').glob('*.csv'):
    if str(df_path).find('MSER') != -1:
        mc = calc_miss_chance(df_path)
        print(f"{df_path.stem} chance is {mc:.3f}%")

MSER+resnet18+256+5 chance is 2.005%


In [155]:
df_orig = pd.read_csv('../../pipeline_results/MI1+resnet101+1024+5.csv', index_col=None)
df_orig = df_orig.drop(['1','2','3','5','6','7'], axis=1)
df_orig['4'] = df_orig['4'].str.replace('S0010', 'S010')
df_orig['8'] = df_orig['8'].str.replace('S0010', 'S010')

df = pd.DataFrame()
df['score'] = df_orig['0'].astype(np.float)
df['index'] = df_orig['4'].str[4:31]
df['id_actual'] = df_orig['4'].str[8:17]
#     df['class_actual'] = df_orig['4'].str[9:13].astype(np.int)
#     df['distinct_actual'] = df_orig['4'].str[15:17].astype(np.int)
#     df['sample_actual'] = df_orig['4'].str[19:22].astype(np.int)
# df['angle_actual'] = df_orig['4'].str[28:31].astype(np.int)
df['index_actual'] = df_orig['4'].str[38:].astype(np.int)

#     df['phone_predicted'] = df_orig['8'].str[6:7].astype(np.int)
df['id_predicted'] = df_orig['8'].str[8:17]
#     df['class_predicted'] = df_orig['8'].str[9:13].astype(np.int)
#     df['distinct_predicted'] = df_orig['8'].str[15:17].astype(np.int)
#     df['sample_predicted'] = df_orig['8'].str[19:22].astype(np.int)

In [285]:
df_res = pd.DataFrame()
df_res['index'] = df['index'].unique()
df_res = df_res.set_index('index')
df_res['predicted'] = ""
df_res['actual'] = ""

In [ ]:
fill_value = 100

un = df['index'].unique()
#pbar = tqdm(np.arange(len(un)), total=len(un), desc='Predicting')
for ci, unique_index in enumerate(un, start=1):
    df_s = df[df['index'] == unique_index]
    
    df_piv = df_s.pivot_table(index=['index', 'id_actual', 'id_predicted'], columns='index_actual', values='score', aggfunc=lambda x: [s for s in x], fill_value=fill_value)
    scores = df_piv.to_numpy().T
    df_ref = df_piv.reset_index()
    #df_ins = df_piv.copy()
    for i, score in enumerate(scores, start=1):
        score = [s if s != fill_value else [fill_value] for s in score]
        metrics = [
            (si, len(x), np.mean(x), sum(x), np.median(x)) for si, x in enumerate(score)
        ]
        metric_len = max(metrics, key=lambda x: x[1])[0]
        metric_mean = min(metrics, key=lambda x: x[2])[0]
        metric_sum = min(metrics, key=lambda x: x[3])[0]
        metric_median = min(metrics, key=lambda x: x[4])[0]

        m = mode([metric_len, metric_len, metric_mean, metric_sum, metric_median]).mode[0]
        fill = [False] * len(score)
        fill[m] = True
        df_piv[i] = fill
    guess_index = df_piv[list(df_piv.columns)].sum(axis=1).argmax()
#     break
    df_res.at[unique_index, 'predicted'] = df_piv.index[guess_index][2]
    df_res.at[unique_index, 'actual'] = df_piv.index[guess_index][1]
    if ci % 1000 == 0:
        print(ci)
#         break
    #pbar.update()
#pbar.close()

1000
2000
3000
4000
5000
6000
7000
8000


In [325]:
df_piv

index_actual                                           1      2      3   \
index                       id_actual id_predicted                        
Ph3_P0630_D01_S001_C1_az200 P0630_D01 P0006_D01     False  False  False   
                                      P0008_D01     False  False  False   
                                      P0028_D01     False  False  False   
                                      P0033_D01     False  False  False   
                                      P0036_D01     False  False  False   
...                                                   ...    ...    ...   
                                      P0863_D01     False  False  False   
                                      P0870_D01     False  False  False   
                                      P0876_D01     False   True  False   
                                      P0879_D01     False  False  False   
                                      P0891_D01     False  False  False   

index_actual                                           4      5      6   \
index                       id_actual id_predicted                        
Ph3_P0630_D01_S001_C1_az200 P0630_D01 P0006_D01     False  False  False   
                                      P0008_D01     False  False  False   
                                      P0028_D01     False  False  False   
                                      P0033_D01     False  False  False   
                                      P0036_D01     False  False  False   
...                                                   ...    ...    ...   
                                      P0863_D01     False  False  False   
                                      P0870_D01     False  False  False   
                                      P0876_D01     False  False  False   
                                      P0879_D01     False  False  False   
                                      P0891_D01     False  False  False   

index_actual                                           7      8      9   \
index                       id_actual id_predicted                        
Ph3_P0630_D01_S001_C1_az200 P0630_D01 P0006_D01     False  False  False   
                                      P0008_D01     False  False  False   
                                      P0028_D01     False  False  False   
                                      P0033_D01     False  False  False   
                                      P0036_D01     False  False  False   
...                                                   ...    ...    ...   
                                      P0863_D01     False  False  False   
                                      P0870_D01     False  False  False   
                                      P0876_D01     False  False  False   
                                      P0879_D01     False  False  False   
                                      P0891_D01     False  False  False   

index_actual                                           10     11     12  \
index                       id_actual id_predicted                        
Ph3_P0630_D01_S001_C1_az200 P0630_D01 P0006_D01     False  False  False   
                                      P0008_D01     False  False  False   
                                      P0028_D01     False  False  False   
                                      P0033_D01     False  False  False   
                                      P0036_D01     False  False  False   
...                                                   ...    ...    ...   
                                      P0863_D01     False  False  False   
                                      P0870_D01     False  False  False   
                                      P0876_D01     False  False  False   
                                      P0879_D01     False  False  False   
                                      P0891_D01     False  False  False   

index_actual                                           13     14     15     16  
index          

In [311]:
df_res['match'] = df_res['predicted'] == df_res['actual']

In [313]:
df_res[df_res['match'] == False]
df_res['match'].value_counts()

True     46278
False     4697
Name: match, dtype: int64

In [315]:
df_res

,predicted,actual,match
index,,,
Ph3_P0630_D01_S001_C1_az200,P0006_D01,P0630_D01,False
Ph1_P0237_D01_S001_C3_az100,P0004_D01,P0237_D01,False
Ph2_P0452_D01_S001_C2_az020,P0003_D02,P0452_D01,False
Ph3_P0323_D01_S001_C2_az060,P0003_D02,P0323_D01,False
Ph1_P0168_D01_S001_C3_az160,P0003_D02,P0168_D01,False
...,...,...,...
Ph3_P0200_D01_S001_C3_az280,,,True
Ph2_P0554_D01_S001_C2_az260,,,True
Ph2_P0234_D01_S001_C3_az160,,,True


In [209]:
from scipy.stats import mode

5